## Description
Fashion_MNIST data and description is available on github: https://github.com/zalandoresearch/fashion-mnist <br>

It consists of a training set of 60,000 examples and a test set of 10,000 examples. Each example is a 28x28 grayscale image, associated with a label from 10 classes as given below:

**Label 	Description** <br>
0 	T-shirt/top <br>
1 	Trouser <br>
2 	Pullover <br>
3 	Dress <br>
4 	Coat <br>
5 	Sandal <br>
6 	Shirt <br>
7 	Sneaker <br>
8 	Bag <br>
9 	Ankle boot <br>

**Points to be Noted**
1. Each image is of size 28x28 = 784 pixels.
2. Grayscale images have a total of 255 shades/pixels ranges from 0 to 255 to define how much dark or light that pixel will be. 0 = Maximum Dark; 255 = Maximum White.
3. Each row is a separate image and column as labels (0-9).

## Callbacks
Used for earty stopping of model. for eg. stopping on a apecific onsition like when Loss printed in the logs is less than a specif value or early stopping when model is not able to reduce the loss further 

In [1]:
#!pip install -U tensorflow==2.0.0-beta1  # to install tensorflow 2.0-beta1 in Kaggle notebook
# Compile and run with TensorFlow 2.0 
import numpy as np
import tensorflow as tf
print(tf.__version__)

# create a callback class
class myCallback(tf.keras.callbacks.Callback):  
    def on_epoch_end(self, epoch, logs=None):  # on_epoch_end is a standard function defined in tensorflow doc & called at the end of epoch  
        if(logs.get('loss')<0.3):   # get the 'loss' output from printed logs
          print("\nReached 70% accuracy so cancelling training!")
          self.model.stop_training = True    # standard flag used to stop the model when abov condition meets

On_epoch_end_callback = myCallback()     # Instantiate the class

mnist = tf.keras.datasets.fashion_mnist
# Download Fashion mnist data from server
(x_training_images, y_training_labels), (x_test_images, y_test_labels) = mnist.load_data()

# Normalize the input images to range between 0 to 1
x_training_images = x_training_images/255.0
x_test_images = x_test_images/255.0

#Define the model by adding layers
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation=tf.nn.relu),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])
#Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
# Train the model
model.fit(x_training_images, y_training_labels, epochs=10, callbacks=[On_epoch_end_callback])


2.0.0-rc0
4423680/4422102 [==============================] - 4s 1us/step
Train on 60000 samples
Epoch 1/10
60000/60000 [==============================] - 22s 360us/sample - loss: 0.4711
Epoch 2/10
60000/60000 [==============================] - 20s 337us/sample - loss: 0.3606
Epoch 3/10
60000/60000 [==============================] - 21s 344us/sample - loss: 0.3231
Epoch 4/10
59872/60000 [============================>.] - ETA: 0s - loss: 0.2973
Reached 70% accuracy so cancelling training!
60000/60000 [==============================] - 21s 343us/sample - loss: 0.2975


## Another Example of callback: Model will stop training when Loss stops reducing

In [8]:
# Code compile and run with Tsnorflow 2.0
import numpy as np
import tensorflow as tf
print(tf.__version__)

# Callback Class
class EarlyStoppingAtMinLoss(tf.keras.callbacks.Callback):
  """Stop training when the loss is at its min, i.e. the loss stops decreasing.
  Arguments:
      patience: Number of epochs to wait after min has been hit. After this
      number of no improvement, training stops.
  """
# This method is called when class in Instantiated
  def __init__(self, patience=0):
    super(EarlyStoppingAtMinLoss, self).__init__()

    self.patience = patience

    # best_weights to store the weights at which the minimum loss occurs.
    self.best_weights = None

# This method is called at the begining of training  
  def on_train_begin(self, logs=None):
    # The number of epoch it has waited when loss is no longer minimum.
    self.wait = 0
    # The epoch the training stops at.
    self.stopped_epoch = 0
    # Initialize the best as infinity.
    self.best = np.Inf

# This method is called on every end of EPOCH     
  def on_epoch_end(self, epoch, logs=None):
    current = logs.get('loss')
    if(current < self.best):
      self.best = current
      self.wait = 0
      # Record the best weights if current results is better (less).
      self.best_weights = self.model.get_weights()
    else:
      self.wait += 1
      if self.wait >= self.patience:
        self.stopped_epoch = epoch
        self.model.stop_training = True   # early stopping of model
        print('Restoring model weights from the end of the best epoch.')
        self.model.set_weights(self.best_weights)

# This method is called when training is finished either of its own or by early stopping        
  def on_train_end(self, logs=None):
    if self.stopped_epoch > 0:
      print('Epoch %05d: early stopping' % (self.stopped_epoch + 1))

Early_stop_callback = EarlyStoppingAtMinLoss()     # Instantiate the class

mnist = tf.keras.datasets.fashion_mnist
# Downloaing Fashion Mnist data
(x_training_images, y_training_labels), (x_test_images, y_test_labels) = mnist.load_data()

# Normalize the data in the range 0 to 1
x_training_images = x_training_images/255.0
x_test_images = x_test_images/255.0
#Define the model
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation=tf.nn.relu),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
model.fit(x_training_images, y_training_labels, epochs=100, callbacks=[Early_stop_callback])

2.0.0-beta1
Train on 60000 samples
Epoch 1/100
60000/60000 [==============================] - 7s 116us/sample - loss: 0.4776
Epoch 2/100
60000/60000 [==============================] - 6s 107us/sample - loss: 0.3603
Epoch 3/100
60000/60000 [==============================] - 6s 105us/sample - loss: 0.3214
Epoch 4/100
60000/60000 [==============================] - 6s 107us/sample - loss: 0.2969
Epoch 5/100
60000/60000 [==============================] - 6s 107us/sample - loss: 0.2821
Epoch 6/100
60000/60000 [==============================] - 6s 105us/sample - loss: 0.2667
Epoch 7/100
60000/60000 [==============================] - 6s 107us/sample - loss: 0.2531
Epoch 8/100
60000/60000 [==============================] - 6s 108us/sample - loss: 0.2404
Epoch 9/100
60000/60000 [==============================] - 7s 117us/sample - loss: 0.2323
Epoch 10/100
60000/60000 [==============================] - 6s 108us/sample - loss: 0.2215
Epoch 11/100
60000/60000 [==============================] - 6s 1